In [ ]:
# https://www.kf.or.kr/kfEng/main.do

In [1]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.firefox.options import Options
 
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'kf_or_kr_publication'
site_name = 'Korea Foundation (Republic of Korea)'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)


# Driver Connection      
options = webdriver.ChromeOptions()
options.add_argument('--headless')
browser = webdriver.Chrome(f'{project_path}/chromedriver', options = options )
browser.maximize_window() 
    
# Firefox driver settings
# options = Options()
# options.headless = True
# browser = webdriver.Firefox(options=options)
# browser.maximize_window()
    
# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_fetch_cat_url = 0

publish_source = 'kf.or.kr'
country = 'China'   # check country
language = 'English'
images_path = []


def button_wait(xpath):
    e = WebDriverWait(browser, 40).until(ec.visibility_of_element_located((By.XPATH, xpath)))
    e.click()
    
    
def text_wait(xpath,text):
    e = WebDriverWait(browser, 30).until(ec.visibility_of_element_located((By.XPATH, xpath)))
    e.send_keys(text)
    
    
def text_wait_enter(xpath,text):
    e = WebDriverWait(browser, 30).until(ec.visibility_of_element_located((By.XPATH, xpath)))
    e.send_keys(text)
    e.send_keys(Keys.ENTER)
    
    
def get_inner_html(xpath):
    try:
        e = WebDriverWait(browser, 20).until(ec.visibility_of_element_located((By.XPATH, xpath)))
        return e.get_attribute('innerHTML')
    except:
        return ''
    
def get_data(xpath):
    try:
        return WebDriverWait(browser, 20).until(ec.visibility_of_element_located((By.XPATH, xpath))).text
    except:
        return ''

home_page = c.download_page('https://www.kf.or.kr/kfEng/main.do?langTy=ENG')
home_page = c.scrap('<ul\s*class="dep01">(.*?)<ul\s*class="dep02">', home_page)

for _ in home_page.split('<li><a href=')[1:2]:  #-1
    cat_url = c.scrap('"(.*?)"', _)   

    if 'http' not in cat_url:
        cat_url = 'https://www.kf.or.kr' +  cat_url
        
    logger.info(f'Fetching cat url  {cat_url}\n')
    print(cat_url)
#     continue
#-----------------------from here selenium is used---------------------------------------------------------------------,

    try:
        browser.get(cat_url)
    except:
        logger.info('Unable to fetch home page\n')
        broswer.quit()
        exit()
    time.sleep(3)
    cat_page = browser.page_source
    if cat_page.startswith('Unable to fetch'):
        logger.info(cat_page) # writes error message with error code
        unable_to_fetch_cat_url += 1
        continue    

    row_count = len(browser.find_elements_by_xpath('//*[@id="cntntsView"]/div[2]/div[2]/div'))
    print(row_count)  
    for i in range(1,row_count+1): 
        
#         source_link= browser.find_element_by_xpath("//*[@id=\"cntntsView\"]/div[2]/div[2]/div["+str(i)+"]/a[contains(@onclick,'detailView')]").click()
        button_wait("//*[@id=\"cntntsView\"]/div[2]/div[2]/div["+str(i)+"]/a")  # esmai wah paticular article ka x path leyga n usko click karwayega
        time.sleep(3)
#         logger.info(f'Fetching {source_link}\n')
#         source_headline = get_data('//*[@id="dgtlModifyForm"]/div[1]')
#         source_content = get_data('//*[@id="dgtlModifyForm"]/div[1]/div')
        page = browser.page_source
        if page.startswith('Unable to fetch'):     
            logger.info(page) # writes error message with error code
            unable_to_fetch_article_url += 1
            continue    

        source_headline = c.scrap('<h3>(.*?)</h3>', page)
        source_headline = re.sub('&.*?;','',source_headline)
        # skip if headline not found
        if not source_headline:
#             logger.info(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue
            
         # Date and time
        pub_date, publish_time = '', ''
        publish_time = current_time
        try:
            date_time_str = c.scrap('<li><strong>Date</strong>(.*?)</li>', page).strip()
            date_time_str = re.sub('[^\w+]', '', date_time_str)  
            date_time_obj = datetime.strptime(date_time_str, '%b%d%Y')
            ist_date_time = date_time_obj - timedelta(hours = 3,minutes = 30)  
            ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
            pub_date = ist_date_time[:10]
            publish_time = ist_date_time[11:]
            
        except:
            pass
        
        # skip null date
        if not pub_date:
#             logger.info(f'Skipping due to date {source_link}\n')            
            skipped_due_to_date += 1
            continue

        # break if date is not today's date
#         if pub_date != created_on:
#             break    


        # source_content         
        source_content= c.scrap('<div\s*class="bbsV_cont">(.*?)<div\s*class="sub_container\s*ac\s*webzine">',page)   
#         if not source_content:
#             source_content= c.scrap('div\s*class="post-data-container">(.*?)<div\s*class="m-b-',page)

        source_content = re.sub('&.*?;','',source_content)
        source_content = c.strip_html(source_content)  
        if not source_content:
#             logger.info(f'Skipping due to content {source_link}\n')            
            skipped_due_to_content += 1
            continue

        
        journalist =c.scrap("'author-name':'(.*?)'",page)

        if not journalist: journalist = 'NA'

        # current date and time 00
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []
    
        # storing the above data in a dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":'',
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content,
                        "pdf_url": pdf_url,
                        "pdf_name": pdf_name,
                        "pdf_path":pdf_path
                    }
#         cl_data.insert_one(clientdata)  
        no_of_data += 1


logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch cat url: {unable_to_fetch_cat_url}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'country: {country}\n')
logger.info(f'language: {language}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')
        

https://www.kf.or.kr/kfEng/na/ntt/selectDgtlInfoList.do?dgtlType=N&mi=2112
8


TimeoutException: Message: 


'jfdjv'